# Notebook E-tivity 3 CE4021 Task 2

Student name: Sean Meade

Student ID: 10128921

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [2]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

First I created a function for making a list of the words from the string that represents an email. 

In [3]:
def list_of_words_from_email(email):

    """
    Takes in a string email and produces the words of that email as a list of strings.
    """

    # split the email into a list of words
    words = email.split()
    
    return words

Then a function that takes in a list of emails and counts the words in it. This is used to find the number of words in the test data for the given spam or ham emails.

In [4]:
def dict_count_words_in_emails(words, emails):

    """
    Takes in a list or set of strings (words) and checks how many words are in the list of strings representing emails (emails).

    It then returns a dictionary whose keys are the words and the values are the number of times that word appeared in the emails.
    """

    words_count = {}

    # for every word in our training data
    for word in words:
        # Create an entry in the dictionary
        words_count[word.lower()] = 0
        # Then loop through the emails
        for email in emails:
            # if the word is in the email
            if word.lower() in email:
                # add the word count to its corresponding dictionary entry
                words_count[word.lower()] += email.count(word)
    
    return words_count

Next I use the output of the last function to calculate the probability the email is in that category.

$$ \mathbb{P}(word|S) = \frac{number~of~spam~emails~containing~the~word~+~1}{number~of~spam~emails~+~2} $$

$$ \mathbb{P}(word|H) = \frac{number~of~ham~emails~containing~the~word~+~1}{number~of~ham~emails~+~2} $$

I use Laplace smoothing to account for words in the test emails that are not in the training emails. This is done by adding 1 to the numerator and 2 to the denominator. So instead of an untrained words working out to a probability of zero it will work out at 1 over the number of emails in that category.

In [5]:
def prob_of_word_in_emails(word_count, emails):

    """
    Takes in a dictionary whose keys are the words and the values are the number of 
    times that word appeared in the emails (word_count) and a list of strings that
    represent emails (emails). Both have to be for the same classification (ham or spam).

    It returns a dictionary whose keys are the words and the values are the probability
    that word will be in an email given that the email is the classification.
    """

    # a dictionary that will contain the probabilities for each word for a given category
    word_prob = {}
    # number of emails
    email_count = len(emails)
    # using the word and it's count
    for word, count in word_count.items():
        # calculate the probability that the word is spam or hame
        word_prob[word] = (count + 1)/(email_count + 2)
    
    return word_prob

Then work out the probability of an email being spam or ham based on the training data.

$$ \mathbb{P}(S) = \frac{number~of~spam~emails}{total~number~of~emails} $$
$$ \mathbb{P}(H) = \frac{number~of~ham~emails}{total~number~of~emails} $$

In [6]:
def probability_of_spam_and_ham(spam_emails, ham_emails):
    """
    This function takes lists of stings representing the spam (spam_emails) and ham emails (ham_emails)
    and returns dictionary where the keys are the classification and the values are the probability that 
    an email will be ham or spam.
    """

    # a dictionary to hold the returned probabilities
    probs = {}

    # total number of emails
    total_email_count = len(spam_emails) + len(ham_emails)

    # then work out the probabilities by the number of emails in the category divided by the total number of emails 
    probs["prob_email_spam"] = len(spam_emails)/total_email_count
    probs["prob_email_ham"] = len(ham_emails)/total_email_count

    return probs

# Calculate and save the probability an email is spam and ham
prob_of_spam_and_ham = probability_of_spam_and_ham(previous_spam, previous_ham)
P_S = prob_of_spam_and_ham["prob_email_spam"]
P_H = prob_of_spam_and_ham["prob_email_ham"]

And as expected the probability of an email being spam plus the probability of an email being ham is equal to 1, because they are the only two possible outcomes.

In [7]:
P_S + P_H

1.0

This next one uses the list_of_words_from_email function on a list of emails.

In [8]:
def list_of_words_from_emails(emails):
    """
    takes in a list of strings representing emails and returns a list of all the words
    """

    # a list to contain the returned list of words
    words_in_emails = []

    # loop through emails
    for email in emails:
        # split the email into words and add them to returned list
        words_in_emails += list_of_words_from_email(email)

    return words_in_emails

Then tying it all together into one function that returns the probability of each word for that category.

In [9]:
def prob_of_words_for(emails):
    """
    Takes in a list of strings representing emails (either ham or spam) and returns a dictionary where the keys 
    are the words and the values are the probability of each word being in an email of the input classification 
    (either ham or spam).
    """

    words = list_of_words_from_emails(emails)
    word_count = dict_count_words_in_emails(words, emails)
    word_probs = prob_of_word_in_emails(word_count, emails)

    return word_probs

Finally we tie everything together we can work out the probability an email is spam given the words on the email:

$$ \mathbb{P}(S|x_1,...,x_i) \approx \frac{\mathbb{P}(S)\prod_{i=1}^{n}\mathbb{P}(x_i|S)}{\mathbb{P}(S)\prod_{i=1}^{n}\mathbb{P}(x_i|S)~+~\mathbb{P}(H)\prod_{i=1}^{n}\mathbb{P}(x_i|H)} $$


In [10]:
def prob_of_email_being_spam(previous_spam, previous_ham, test_emails, threshold=0.6):

    """
    A function that takes in lists of strings representing test emails for spam (previous_spam) 
    and ham (previous_ham), a dictionary that contains two keys ("spam" and "ham") whose values 
    are a list of stings that represent emails., and an optional float number for the probability 
    threshold (threshold).

    Returns a dictionary whose keys represent the classification (ham or spam) and the values are 
    another dictionary whose keys are a string that is the test emails and the value is a list 
    containing a boolean that represents if the classifier sees the email as spam or not and a
    float representing the probability the email is spam.
    """

    # the two categories
    categories = ["spam", "ham"]

    # the output dictionary that holds the emails, the calculated probability, and the boolean of whether the function classes it as spam or not
    email_is_spam = {"spam": {}, "ham": {}}

    # all unique words in the training data
    training_words = set(list_of_words_from_emails(previous_ham) + list_of_words_from_emails(previous_spam))

    # the probability an email could be spam or ham
    prob_of_spam_and_ham = probability_of_spam_and_ham(previous_spam, previous_ham)
    P_S = prob_of_spam_and_ham["prob_email_spam"]
    P_H = prob_of_spam_and_ham["prob_email_ham"]

    # Dictionaries of the probability of each word given the category
    P_w_S = prob_of_words_for(previous_spam)
    P_w_H = prob_of_words_for(previous_ham)

    # for each category
    for cat in categories:
        # and for each email in the test data for that category
        for email in test_emails[cat]:
            # get the unique words in the email
            words_in_email = set(list_of_words_from_email(email))
            # set new variables to hold the product of all the probabilities of each word given its spam or ham
            P_wi_S = P_S
            P_wi_H = P_H
            # for each word in the email
            for word in words_in_email:
                # and if that word is in the training data
                if word in training_words:
                    # check to see its in the spam data
                    try:
                        # and multiply it in
                        P_wi_S *= P_w_S[word]
                    except:
                        # otherwise use the laplace to add the probability so it doesn't multiply to zero
                        P_wi_S *= 1/(2 + len(previous_spam))
                    # check to see its in the ham data
                    try:
                        # and multiply it in
                        P_wi_H *= P_w_H[word]
                    except:
                        # otherwise use the laplace to add the probability so it doesn't multiply to zero
                        P_wi_H *= 1/(2 + len(previous_ham))
            # work out the probability that the email is spam
            probability_its_spam = P_wi_S/(P_wi_S + P_wi_H)
            # add the email to the output
            email_is_spam[cat][email] = [(probability_its_spam > threshold), probability_its_spam]

    return email_is_spam

# run the function with the data given
prob_of_email_being_spam(previous_spam, previous_ham, new_emails)

{'spam': {'renew your password': [True, 0.9174311926605504],
  'renew your vows': [True, 0.6493506493506493]},
 'ham': {'benefits of our account': [True, 0.7941550190597204],
  'the importance of physical activity': [False, 0.026092764998121312]}}

The last output shows whether the spam filter categorizes it as spam or not and the probability calculated for it. You will notice that the spam emails and the second ham email are predicted correctly. However if the filter is set to anything less 0.79 then this email will be classed as spam even though it's classed as ham.

In [11]:
previous_spam

['send us your password',
 'review our website',
 'send your password',
 'send us your account']

'benefits of our account' has two words ('account' and 'our) in the spam training data. 

In [12]:
previous_ham

['Your activity report', 'benefits physical activity', 'the importance vows']

'benefits of our account' only has one word ('benefits') in the ham training data.

This has thrown off the calculation. To fix this using more data would probably even things out a bit better.

Finally I create a class with the above so it can then use the new_emails to inform future predictions:

In [13]:
class spam_filter():
    """
    A class that takes in two lists of strings, test ham emails (previous_ham) and test 
    spam emails (previous_spam), when initialized.
    """

    # Set values for test_ham and test_spam when initializing an instance of the class
    def __init__(self, previous_ham, previous_spam):
        
        self.test_ham = previous_ham
        self.test_spam = previous_spam

    def add_new_emails_to_test(self, new_emails):
        """
        A function that takes in a dictionary that contains two keys ("spam" and "ham") whose values
        are a list of stings that represent emails. It checks to see if the emails are 
        new to the test data and if not adds them
        """

        # loop through emails
        for email in new_emails["ham"]:
            # check if they are in the test data
            if not email in self.test_ham:
                # add the email if they aren't
                self.test_ham.append(email)
        # loop through emails
        for email in new_emails["spam"]:
            # check if they are in the test data
            if not email in self.test_spam:
                # add the email if they aren't
                self.test_spam.append(email)


    def predict_new_emails(self, new_emails):
        """
        A function that takes in a dictionary that contains two keys ("spam" and "ham") whose values
        are a list of stings that represent emails.
        """

        # run the prediction using the test data and the new emails
        predictions = prob_of_email_being_spam(self.test_spam, self.test_ham, new_emails)
        
        # check to see if there are any new emails to add to test data
        self.add_new_emails_to_test(new_emails)
        # return the prediction
        return predictions

# initialize a new spam filter and pass in the training data
spam_filter_instance = spam_filter(previous_ham, previous_spam)

In [14]:
# predict the new_emails
spam_filter_instance.predict_new_emails(new_emails)

{'spam': {'renew your password': [True, 0.9174311926605504], 'renew your vows': [True, 0.6493506493506493]}, 'ham': {'benefits of our account': [True, 0.7941550190597204], 'the importance of physical activity': [False, 0.026092764998121312]}}


In [15]:
# check to see the new_emails are added to the ham data
spam_filter_instance.test_ham

['Your activity report',
 'benefits physical activity',
 'the importance vows',
 'benefits of our account',
 'the importance of physical activity']

In [16]:
# check to see the new_emails are added to the spam data
spam_filter_instance.test_spam

['send us your password',
 'review our website',
 'send your password',
 'send us your account',
 'renew your password',
 'renew your vows']

In [18]:
spam_filter_instance.predict_new_emails({'spam': ['renew your password', 'your pin has expired'], 'ham': ['benefits of our account', 'this weeks newsletter']})

{'spam': {'renew your password': [True, 0.9830166648975692], 'your pin has expired': [True, 0.8630136986301369]}, 'ham': {'benefits of our account': [False, 0.15423935686951099], 'this weeks newsletter': [False, 0.5454545454545454]}}


In [20]:
# check to see that only new emails are added to the ham test data
spam_filter_instance.test_ham

['Your activity report',
 'benefits physical activity',
 'the importance vows',
 'benefits of our account',
 'the importance of physical activity',
 'this weeks newsletter']

In [19]:
# check to see that only new emails are added to the spam test data
spam_filter_instance.test_spam

['send us your password',
 'review our website',
 'send your password',
 'send us your account',
 'renew your password',
 'renew your vows',
 'your pin has expired']

<hr style=\"border:2px solid gray\"> </hr>

## Reflection

Based on Jason Colemans comment on my forum post and going through the code a second time I realized that the remove_untrained_words function was not needed. This led to reducing my code and still functioning correctly.

I will describe each function as they are introduced

### list_of_words_from_email

This function takes in a string (representing an email) and then outputs a list of strings representing the words of the email.

### dict_count_words_in_emails

This function takes in a list or set of strings (words) and checks how many words are in the list of strings representing emails (emails).

It then returns a dictionary whose keys are the words and the values are the number of times that word appeared in the emails.

### prob_of_word_in_emails

Takes in a dictionary whose keys are the words and the values are the number of times that word appeared in the emails (word_count) and a list of strings that represent emails (emails). Both have to be for the same classification (ham or spam).

It returns a dictionary whose keys are the words and the values are the probability that word will be in an email given that the email is the classification.

### probability_of_spam_and_ham

This function takes lists of stings representing the spam (spam_emails) and ham emails (ham_emails) and returns dictionary where the keys are the classification and the values are the probability that an email will be ham or spam.

### list_of_words_from_emails

This function takes in a list of strings and loops through them. Using the list_of_words_from_email function it returns a list of strings representing all the words in the emails.

### prob_of_words_for

Takes in a list of strings representing emails (either ham or spam) and returns a dictionary where the keys are the words and the values are the probability of each word being in an email of the input classification (either ham or spam).

### prob_of_email_being_spam

A function that takes in lists of strings representing test emails for spam (previous_spam) and ham (previous_ham), a dictionary that contains two keys ("spam" and "ham") whose values are a list of stings that represent emails., and an optional float number for the probability threshold (threshold).

It returns a dictionary whose keys represent the classification (ham or spam) and the values are another dictionary whose keys are a string that is the test emails and the value is a list containing a boolean that represents if the classifier sees the email as spam or not and a float representing the probability the email is spam.

### class spam_filter

Finally a class that takes the test spam emails and test ham emails on initialization. Then for every time it predicts new emails it will add them to the test data helping to improve the classifier.